In [2]:
import pyarrow as pa
import pyarrow.dataset as pds
import pyarrow.compute as pc
from pyarrow import parquet
import polars as pl
import numpy as np

from pathlib import Path
from datetime import datetime

scia_base = Path("/Users/davidenicoli/Local_Workspace/Datasets/SCIA/giornaliere/")
tmax_path = scia_base / "massime" / "db" / "series.parquet"
tmin_path = scia_base / "minime" / "db" / "series.parquet"

In [14]:
dicta = pa.array(["T_MIN", "T_MAX"], pa.string())

In [16]:
tmin = parquet.read_table(tmin_path, filters = [("date", ">=", datetime(2000,1,1))]).rename_columns(["value", "date", "identifier"])
tmin = tmin.append_column("variable", pa.DictionaryArray.from_arrays(pa.array([0]*tmin.num_rows, pa.uint8()), dicta))
tmax = parquet.read_table(tmax_path, filters = [("date", ">=", datetime(2000,1,1))]).rename_columns(["value", "date", "identifier"])
tmax = tmax.append_column("variable", pa.DictionaryArray.from_arrays(pa.array([1]*tmax.num_rows, pa.uint8()), dicta))

ds = pa.concat_tables([tmin, tmax]) #.cast(pa.schema([("value", pa.float32()), ("date", pa.date32()), ("identifier", pa.uint32()), ("variable", pa.dictionary(pa.uint8(), pa.string()))]))
part = pds.partitioning(schema=pa.schema([("variable", pa.dictionary(pa.uint8(), pa.string()))]), flavor="hive")

In [18]:
pds.write_dataset(
    ds, scia_base / "python_dataset",
    format="parquet",
    partitioning=part
)